import everything

In [ ]:
import os
import glob
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
import types
warnings.filterwarnings("ignore")
import sys, time
import pickle
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib import mlab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
from sklearn.decomposition import KernelPCA
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter
import imageio



if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    %run ../BehavioralPaper/ImmTrd.ipynb
    os.chdir(CWD)
    # PARAMETERS (used if the pickles don't exist)
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }  

    if OS()=='Linux':
        root="/data"
    elif OS()=='Windows':
        root="C:\\DATA\\"
    else:
        root="/Users/davidrobbe/Documents/Data/"
        
    logging.getLogger().setLevel(logging.ERROR)

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

# part 1:

# DEFINITIONS

### If you don't know, move to part 2

Function to add treadmill image to the left of x axis

In [ ]:
def convert_and_open_treadmill():
    pngPath=os.path.join(os.path.expanduser('~'),'Pictures','TrEaDmIlL.png')

    RepoPath=os.path.split(os.getcwd())[0]
    svgPath=glob.glob(os.path.join(RepoPath,'BehavioralPaper','*.svg'))[0]
    if not os.path.exists(pngPath):
        os.system(f'inkscape -z -f {svgPath} -e {pngPath}')
        print('png created')

    treadmill = plt.imread(pngPath)
    treadmill2 =scipy.ndimage.rotate(treadmill,90,order=0, mode='constant', prefilter=False)
    return treadmill,treadmill2

In [ ]:
def add_treadmill_to_axis(ax, path, Xextention=13):
    
    x1,x2=ax.get_xlim()
    ax.set_xlim([x1-Xextention,x2])
    insetAx=inset.inset_axes(ax,width="5%", height="100%", loc=6)
    insetAx.imshow(plt.imread(path))
    insetAx.set_axis_off()
    return ax

plotting several consecutive trials

In [ ]:
def plot_consecutive_trajectories(root, session, trials, ax):
    delta=1 #last delta second of position is not recorded
    data=Data(root,session[:6],session,param=param,redoPreprocess=False, saveAsPickle=False);
    data.position_correction()
    try:
        detect_trial_end(data, trials)
    except:
        pass
    fps=data.cameraSamplingRate
    y1=param['treadmillRange'][0]
    y2=param['treadmillRange'][1]

    maxT=0
    for trial in trials:
        time=data.rawTime[trial]+maxT
        maxT=time[-1]+delta
        #plotting position
        ax.plot(time,data.position[trial],'k');
        #plotting entrance time
        ET=data.entranceTime[trial]+data.cameraToTreadmillDelay+time[0]
        if data.entranceTime[trial]!=data.maxTrialDuration[trial]:
            ax.plot(ET,data.position[trial][time[time<ET].argmax()+1],'rx')
        #plotting the goal time
        GT=data.goalTime[trial]+data.cameraToTreadmillDelay+time[0]
        ax.plot(GT,data.position[trial][time[time<GT].argmax()], marker='o',markeredgecolor='r', markerfacecolor='None')
        #plotting the highlight for trials
        x=(time[0], time[0]+data.timeEndTrial[trial]+data.cameraToTreadmillDelay)
        c='g' if trial in data.goodTrials else 'r'
        ax.fill_betweenx(y=(y1,y2),x1=x[0],x2=x[1], color=c, alpha=.2)
    
    ax.set_xlim([-1,time[-1]+1])
    ax.set_ylim([y1-2,y2+2])
    ax.set_yticks([0,50,80])
#     ax.set_xlabel('Time (s)')
    ax.set_ylabel('Position (cm)')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
        
    return ax
    

In [ ]:
if "__file__" not in dir():
    path=os.path.join(os.path.expanduser('~'),'Pictures','TrEaDmIlL2.png')

    session='Rat123_2017_03_17_17_52'
    trials=[50,51,52,53,54,55,56,57,58]

    plt.close('all')
    ax=plt.figure(figsize=(20,3)).add_subplot(111);

    plot_consecutive_trajectories(root, session, trials, ax)
    add_treadmill_to_axis(ax,path,Xextention=12)

In [ ]:
if "__file__" not in dir():
    path=os.path.join(os.path.expanduser('~'),'Pictures','TrEaDmIlL2.png')
    fig=plt.figure(figsize=(20,9),dpi=100)
    naiveAx=fig.add_subplot(411)
    trainedAx=fig.add_subplot(412)
    NaiveStupidAx=fig.add_subplot(413)
    trainedStupidAx=fig.add_subplot(414)

    
    #plotting naive
    session='Rat123_2017_02_14_18_33'
    trials=range(10,18)
    plot_consecutive_trajectories(root, session, trials, naiveAx)
    add_treadmill_to_axis(naiveAx,path,Xextention=8)
    naiveAx.text(x=0, y=0, s=f'{session[:6]} naive')
    #plotting trained
    session='Rat123_2017_03_17_17_52'
    trials=range(50,59)
    plot_consecutive_trajectories(root, session, trials, trainedAx)
    add_treadmill_to_axis(trainedAx,path,Xextention=8)
    trainedAx.text(x=0, y=0, s=f'{session[:6]} trained')
    
    #plotting stupid naive animal
    session='Rat132_2017_05_26_13_34'
    trials=range(121,129)
    plot_consecutive_trajectories(root, session, trials, NaiveStupidAx)
    add_treadmill_to_axis(NaiveStupidAx,path,Xextention=8)
    NaiveStupidAx.text(x=0, y=0, s=f'{session[:6]} naive')

    #plotting stupid trained animal
    session='Rat132_2017_06_09_14_06'
    trials=range(70,78)
    plot_consecutive_trajectories(root, session, trials, trainedStupidAx)
    add_treadmill_to_axis(trainedStupidAx,path,Xextention=8)
    trainedStupidAx.text(x=0, y=0, s=f'{session[:6]} trained')
    
#     fig.savefig('/home/david/Pictures/tst.pdf',format='pdf')
    plt.show()
    plt.close('all')

plotting the trajectories of above example sessions

In [ ]:
def position_aligned_on_entrance_time(root,session,Win):
    data=Data(root,session[:6],session,redoPreprocess=False)
    fs=data.cameraSamplingRate
    delay=data.cameraToTreadmillDelay
    posDict=data.position

    pos=[]
    trialColor=[]
    for trial in posDict:
        et=data.entranceTime[trial]
        position=posDict[trial]
        maxTrial=data.maxTrialDuration[trial]
        try:
            th=int((et+delay)*fs)
        except ValueError:
            logging.warning(f'session {session} trial {trial}: entrance time error')
            continue

        assert isinstance(th,int), 'blown up!'

        try:
            pos.append(posDict[trial][th-int(Win*fs):th+1])
        except IndexError:
            pos.append(posDict[trial][:th+1])

        if trial in data.goodTrials:
            trialColor.append('g') 
        else:
            trialColor.append('r')
        

    winL=stats.mode([len(i) for i in pos],nan_policy='omit')[0][0]
    position =np.ones((winL,len(pos))) * np.nan
    for i,traj in enumerate(pos):
        l=len(traj)
        position[winL-l:,i]=traj
    time=np.linspace(-Win,0,winL)
    return position, time, np.array(trialColor)

def plot_trajectories_aligned_on_entrance_time(root, ax, session, Win):
    position,time,color=position_aligned_on_entrance_time(root,session, Win)
    y1,y2=treadmillRange=param['treadmillRange']
    lines=ax.plot(time,position,'k', lw=.5)
    for i,line in enumerate(lines):
        line.set_color(color[i])
#         line.set_zorder( color[i]=='g')

    histT0,bins0=np.histogram(position[0,color=='g'],30,range=(treadmillRange[0],treadmillRange[1]), density=False)
    histT1,bins1=np.histogram(position[0,color=='r'],30,range=(treadmillRange[0],treadmillRange[1]), density=False)
    
    maxBin=max([histT0.max(),histT1.max()])*2
    ax.barh(bins0[:-1],-histT0/maxBin,height=np.diff(bins0)[0],align='edge',left=-Win, color='g', alpha=.6)
    ax.barh(bins1[:-1],-histT1/maxBin,height=np.diff(bins1)[0],align='edge',left=-Win, color='r', alpha=.6)

    ax.set_xlim([-Win-.5,0])
    ax.set_xticks(np.arange(-Win,.1,.5))
    ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_bounds(ax.get_xticks()[0],ax.get_xticks()[-1])
    ax.set_ylim([y1-2,y2+2])
    ax.set_yticks([0,50,80])

    ax.set_xlabel('Time to entrance time (s)')
    ax.set_ylabel('Position (cm)')

    return ax

In [ ]:
if "__file__" not in dir():
    #the inputs
    session='Rat123_2017_03_17_17_52'
    Win=2
    
    
    fig=plt.figure(figsize=(3,13),dpi=100)
    naiveAx=fig.add_subplot(411)
    trainedAx=fig.add_subplot(412)
    NaiveStupidAx=fig.add_subplot(413)
    trainedStupidAx=fig.add_subplot(414)

    
    #plotting naive
    session='Rat123_2017_02_14_18_33'
    plot_trajectories_aligned_on_entrance_time(root, naiveAx, session, Win)
#     naiveAx.text(x=0, y=0, s=f'{session[:6]} naive')
    #plotting trained
    session='Rat123_2017_03_17_17_52'
    plot_trajectories_aligned_on_entrance_time(root, trainedAx, session, Win)
#     trainedAx.text(x=0, y=0, s=f'{session[:6]} trained')
    
    #plotting stupid naive animal
    session='Rat132_2017_05_26_13_34'
    plot_trajectories_aligned_on_entrance_time(root, NaiveStupidAx, session, Win)
#     NaiveStupidAx.text(x=0, y=0, s=f'{session[:6]} naive')

    #plotting stupid trained animal
    session='Rat132_2017_06_09_14_06'
    plot_trajectories_aligned_on_entrance_time(root, trainedStupidAx, session, Win)
#     trainedStupidAx.text(x=0, y=0, s=f'{session[:6]} trained')
    
#     fig.savefig('/home/david/Pictures/tst.pdf',format='pdf')
    plt.show()
    plt.close('all')

Drawing the reward magnitude

In [ ]:
def plot_reward_progression_schematic(ax, minReward, gt, maxTrial, maxReward, step, color,showYAxis):
    points=[(i,(i-minReward)/(gt-minReward)*.6) for i in np.arange(minReward,gt,step)]
    rewardRatio = lambda x: (-x/(maxReward-gt)+ (maxReward/(maxReward-gt)))

    for x,alpha in points:
        ax.plot( (x,gt), (0,1), alpha=alpha, color=color )

    ax.plot( (gt,gt), (1,0), alpha=1, color=color, lw=2 )
    ax.plot( (gt,maxTrial), (1,rewardRatio(maxTrial)), alpha=1, color=color, lw=2 )
    ax.plot( (maxTrial,maxReward), (rewardRatio(maxTrial),0), ':', alpha=1, color=color, lw=2)

    ax.yaxis.set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.set_xticks([minReward, gt, maxTrial, maxReward])
    ax.tick_params(top='off', bottom='off', left='off', right='off', labelleft='on', labelbottom='on')
    ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
    ax.set_xlim([minReward,maxReward])
    if showYAxis:
        ax.yaxis.set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.set_ylim([0,1])
        ax.set_yticks([0,1])
        ax.set_yticklabels(['zero', 'max'])

    #adding the text
    ax.set_title('Reward\nMagnitude')
    ax.set_xlabel('Entrance Time (s)')
    
    return ax

In [ ]:
if "__file__" not in dir():
    minReward=1.5
    gt=7
    maxTrial=15
    maxReward=20
    step=.8
    color='r'
    showYAxis=False
    ax=plt.figure(figsize=(2,2)).add_subplot(111);

    plot_reward_progression_schematic(ax, minReward, gt, maxTrial, maxReward, step, color,showYAxis)
    
    plt.show()
    plt.close('all')

group ET learning curve

In [ ]:
def plot_dotted_learning_curve(ax, root, animalList, profile, TaskParamToPlot, stop_dayPlot, seed=1):
    Results,_=get_rat_group_statistic(root,
                                      animalList,
                                      profile,
                                      parameter=param,
                                      redo=False,
                                      stop_dayPlot=stop_dayPlot,
                                      TaskParamToPlot=[TaskParamToPlot])

    goalTime=data_fetch(root, animal=animalList[0], profile=profile,
                        PerfParam= [lambda data:data.goalTime[-1]],
                        NbSession=0).values()
    goalTime=list(goalTime)[-1]
    
    x=np.arange(stop_dayPlot)+1
    data=np.array( list( Results[TaskParamToPlot].values() ) )
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    np.random.seed(seed=seed)
    sigma=.1
    
    ax.errorbar(x,y,yerr=abs(yerr-y), ecolor='k', fmt='k-o', markerfacecolor='w',zorder=1)
    for pts,day in zip(data.T,x):
        jitter=np.random.normal(day, sigma, len(pts))
        jitter=np.clip(a=jitter, a_min=day-2*sigma, a_max=day+2*sigma)
        ax.scatter(jitter,pts,s=10,c=range(len(animalList)),cmap='plasma', marker='o',zorder=2)

    ax.set_xlim([x[0]-1,x[-1]+1])
    xtick=[1]
    for i in range(1,stop_dayPlot+1):
        if i%5==0:
            xtick.append(i)
    ax.set_xticks(xtick)
    ax.set_ylim([0,10])
    ax.set_yticks([0,7,10])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
#     ax.grid(b=True, which='major', axis='x')
    ax.hlines(y=goalTime, xmin=x[0], xmax=x[-1], linestyle='--')
    ax.text(x=x[5], y=goalTime[-1], s='goal time', verticalalignment='center', backgroundcolor='w')
#     ax.spines['left'].set_position(('data',-1))

    ax.set_xlabel('Session')
    ax.set_ylabel(TaskParamToPlot)
    
    return ax

In [ ]:
if "__file__" not in dir():

    profile={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'0',
             'Speed':'0',
             'Tag':['ImmobileTreadmill']
             }
    animalList=batch_get_animal_list(root,profile)
    animalList=['Rat121','Rat122','Rat123','Rat124','Rat131','Rat132']

    TaskParamToPlot="percentile entrance time"
    stop_dayPlot =41
    
    plt.close('all')
    fig=plt.figure(figsize=(10,5))
    ax=fig.add_subplot(111);
    
    plot_dotted_learning_curve(ax, root, animalList, profile, TaskParamToPlot, stop_dayPlot)
    ax.set_ylabel('Entrance Times (s)')

plotting goal time change effect

In [ ]:
def plot_goal_time_change(root, Profiles, N, GT, badAnimals, TaskParamToPlot, wspace, gs, fig):
    """
    gs could be either gridSpec or a list of axes with 
    """
    if isinstance(gs,matplotlib.gridspec.SubplotSpec):
        gssub = gs.subgridspec(1, len(Profiles)-1,wspace=wspace)
        axes=[]
    else:
        gssub=False
        axes=gs
    for eventN in range(len(N)-1):
        #getting the data
        _,SessionDict=event_detect(root, Profiles[eventN], Profiles[eventN+1], badAnimals=badAnimals)

        Results,nSessionPre,nSessionPost=event_statistic(root,
                                                         SessionDict,
                                                         parameter=param,
                                                         redo=False,
                                                         TaskParamToPlot=[TaskParamToPlot])

        assert N[eventN]<nSessionPre and N[eventN+1]<nSessionPost,f"fewer sessions available than requested:{Profiles[eventN]['Tag']}"

        data=np.array(list(Results[TaskParamToPlot].values()))
        data=data[:,
                  nSessionPre-N[eventN] : nSessionPre+N[eventN+1]
                 ]

        y=np.nanpercentile(data,50,axis=0)
        yerr=np.nanpercentile(data,(25,75),axis=0)

        #getting the axes
        if gssub:
            ax=fig.add_subplot(gssub[0, eventN])
        else:
            ax=axes[eventN]

        #plotting
        xLabel=list(range(-N[eventN],N[eventN+1]+1))
        xLabel.remove(0)
        ax.errorbar(xLabel,y,yerr=abs(yerr-y), ecolor='k', fmt='k-o', markerfacecolor='w',zorder=1)
        ax.plot([xLabel[0],-1,1,xLabel[-1]],[GT[eventN],GT[eventN],GT[eventN+1],GT[eventN+1]], 'k--', zorder=-5)



        if xLabel[0]<=-10 or xLabel[-1]>=10:
            newLabel=[label for label in xLabel if label%5==0]
            newLabel.extend([1,-1])
            newLabel.extend([xLabel[0],xLabel[-1]])
            newLabel=list(set(newLabel))
            newLabel.sort()
            xLabel=newLabel

        ax.set_xlim([xLabel[0]-.9,xLabel[-1]+.9])
        ax.set_xticks(xLabel)
        ax.spines['bottom'].set_bounds(xLabel[0],xLabel[-1])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        ax.set_ylabel(TaskParamToPlot)
        ax.set_ylim([0,10])
        ax.set_yticks(list(set(GT))+[10])
        ax.vlines(x=0, ymin=ax.get_ylim()[0], ymax=ax.get_ylim()[1]*.9,lw=2, color='m',linestyles='--')

        if gssub:
            axes.append(ax)

    axes[1].set_xlabel('session relative to goal time change')
    axes[0].spines['left'].set_position(('axes',-.05))
    for ax in axes[1:]:
        ax.yaxis.set_visible(False)
        ax.spines['left'].set_visible(False)

    return tuple(axes)

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill']
              }
    profile2={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-GT4']
              }
    profile3={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-GT2']
              }
    profile4={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-BackToGT7']
              }

    #number of sessions to plot
    N=[25,10,10,25]
    #goal times
    GT=[7,4,2,7]
    badAnimals=['RatBAD']
    TaskParamToPlot="percentile entrance time"
    wspace=0.05
    
    
    Profiles=(profile1,profile2,profile3,profile4)
    plt.close('all')
    fig=plt.figure(figsize=(15,4))
    gs=fig.add_gridspec(1,1)[0]
    
    axes=plot_goal_time_change(root, Profiles, N, GT, badAnimals, TaskParamToPlot, wspace, gs, fig)
#     axes=plot_goal_time_change(root, Profiles, N, GT, badAnimals,
#                                TaskParamToPlot='standard deviation of entrance time', wspace=wspace, gs=axes, fig=fig)
    
    axes[0].set_ylabel('Entrance Times (s)')

Drawing task rules 

In [ ]:
def plot_task_rules(fig, gs, hspace, alpha, xmin, xmax, et, GT, maxP):
    gssub = gs.subgridspec(2, 1,hspace=hspace)
    goodAx=fig.add_subplot(gssub[0, 0])
    badAx =fig.add_subplot(gssub[1, 0])
    axes=(goodAx,badAx)
    ET,ET0=et=(9,5)
    
    #good trials
    ######################################
    
    r=np.array([0,1])

    #light on
    goodAx.fill_betweenx(y=r,x1=-1,x2=ET, facecolor='yellow', edgecolor='', alpha=alpha)
    goodAx.fill_betweenx(y=r,x1=-10,x2=-1, facecolor='gray', edgecolor='', alpha=alpha)
    goodAx.fill_betweenx(y=r,x1=ET,x2=30, facecolor='gray', edgecolor='', alpha=alpha)
    goodAx.text(x=-1, y=r.mean(), s='Light ON', verticalalignment='center', fontsize=8)
    goodAx.text(x=ET, y=r.mean(), s='Intertrial, and reward', verticalalignment='center', fontsize=8)

    r+=1
    #motor running
    goodAx.fill_betweenx(y=r,x1=0,x2=ET, facecolor='cyan', edgecolor='', alpha=alpha)
    goodAx.text(x=0, y=r.mean(), s='Motor ON', verticalalignment='center', fontsize=8)

    r+=1
    #beam ignore
    goodAx.fill_betweenx(y=r,x1=0,x2=1.5, facecolor='pink', edgecolor='', alpha=alpha)
    goodAx.text(x=0, y=r.mean(), s='Beam OFF', verticalalignment='center', fontsize=8)

    # r+=1
    #entrance times
    goodAx.fill_betweenx(y=r,x1=GT,x2=15, facecolor='lime', edgecolor='', alpha=alpha)
    goodAx.text(x=GT, y=r.mean(), s='Entrance time range', verticalalignment='center', fontsize=8)

    #Correct trials
    goodAx.text(x=xmin, y=5, s='Correct trial', verticalalignment='center', fontsize=10)



    #bad trials
    ######################################
    r=np.array([0,1])

    #light on
    badAx.fill_betweenx(y=r,x1=-1,x2=maxP, facecolor='yellow', edgecolor='', alpha=alpha)
    badAx.fill_betweenx(y=r,x1=-10,x2=-1, facecolor='gray', edgecolor='', alpha=alpha)
    badAx.fill_betweenx(y=r,x1=maxP,x2=30, facecolor='gray', edgecolor='', alpha=alpha)
    # badAx.text(x=-1, y=r.mean(), s='Light ON', verticalalignment='center', fontsize=8)
    badAx.text(x=maxP, y=r.mean(), s='Intertrial, no reward', verticalalignment='center', fontsize=8)

    r+=1
    #motor running
    badAx.fill_betweenx(y=r,x1=0,x2=maxP, facecolor='cyan', edgecolor='', alpha=alpha)
    # badAx.text(x=0, y=r.mean(), s='Motor ON', verticalalignment='center', fontsize=8)

    r+=1
    #beam ignore
    badAx.fill_betweenx(y=r,x1=0,x2=1.5, facecolor='pink', edgecolor='', alpha=alpha)
    # badAx.text(x=0, y=r.mean(), s='Beam OFF', verticalalignment='center', fontsize=8)

    # r+=1
    # punishment
    badAx.fill_betweenx(y=r,x1=ET0,x2=maxP, facecolor='r', edgecolor='', alpha=alpha)
    badAx.text(x=ET0, y=r.mean(), s='Punishment', verticalalignment='center', fontsize=8,zorder=5)

    r+=1
    #entrance times
    badAx.fill_betweenx(y=r,x1=1.5,x2=GT, facecolor='lime', edgecolor='', alpha=alpha)
    # badAx.text(x=1.5, y=r.mean(), s='Entrance time range', verticalalignment='center', fontsize=8)

    #Correct trials
    badAx.text(x=xmin, y=5, s='Incorrect trial', verticalalignment='center', fontsize=10)

    #common axes
    for i,ax in enumerate(axes):
        ax.vlines(x=GT, ymin=0, ymax=4.4, color='m', linestyles='--', zorder=2)
        ax.vlines(x=et[i], ymin=-5, ymax=i*2, color='k', linestyles='-', zorder=2, lw=.8)
        ax.set_xlim([xmin,xmax])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.yaxis.set_visible(False)
        ax.set_xticks([-1,0,1.5,7,15])
        ax.xaxis.set_major_formatter(FormatStrFormatter('%g'))
        ax.set_ylim([-.1,5])

    goodAx.text(x=GT, y=4.4, s='Goal time', ha='center', fontsize=8, color='m')

    goodAx.set_xticklabels([])

    #adding ET
    goodAx.set_xticks(list(goodAx.get_xticks())+[ET])
    l=[]
    for tick in goodAx.get_xticks():
        if tick==ET:
            l.append('$ET$')
        else:
            l.append('')
    goodAx.set_xticklabels(l)

    badAx.set_xticks(list(badAx.get_xticks())+[ET0])
    l=[]
    for tick in badAx.get_xticks():
        if tick==ET0:
            l.append('$ET$')
        else:
            l.append('%g'%(tick))
    badAx.set_xticklabels(l)

    # badAx.set_xlabel(r'$\longrightarrow$'+'\n'+'Time(s)')
    badAx.set_xlabel('Time (s)')


    # fig.add_artist(matplotlib.patches.FancyArrow(.45, 0, dx=.12, dy=0,length_includes_head=True,
    #                                              head_width=.02, head_length=.01, color='k'))
    return axes

In [ ]:
if "__file__" not in dir():

    alpha=1
    hspace=.4
    xmin,xmax=(-2,16)
    et=(9,5)
    GT=7
    maxP=10 #punishment

    plt.close('all')
    fig=plt.figure(figsize=(5,2),dpi=200)
    gs=fig.add_gridspec(1,1)[0]
    axes=plot_task_rules(fig, gs, hspace, alpha, xmin, xmax, et, GT, maxP)
    plt.show()

# part 2:

# GENERATING THE FIGURE

Parameters

In [ ]:
if "__file__" not in dir():
    # GRID 1 PARAMS
    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'initialSpeed':'0',
             'Speed':'0',
             'Tag':['ImmobileTreadmill']
             }
    animalList1=batch_get_animal_list(root,profile1)
    animalList1=['Rat121','Rat122','Rat123','Rat124','Rat131','Rat132']

    TaskParamToPlot1="percentile entrance time"
    stop_dayPlot1 =30
    #===============================================
    
    # GRID 2 PARAMS
    profile21={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill']
              }
    profile22={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-GT4']
              }
    profile23={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-GT2']
              }
    profile24={'Type':'Good',
              'rewardType':'Progressive',
              'initialSpeed':'0',
              'Speed':'0',
              'Tag':['ImmobileTreadmill-BackToGT7']
              }

    #number of sessions to plot
    N2=[25,10,10,25]
    #goal times
    GT2=[7,4,2,7]
    badAnimals2=['RatBAD']
    TaskParamToPlot2="percentile entrance time"
    wspace2=0.05  
    Profiles2=(profile21,profile22,profile23,profile24)
    #================================================
    
    # GRID 3 PARAMS: entrance time examples
    Win3=2
    #plotting naive
    session3goodNaive    ='Rat123_2017_02_14_18_33'
    #plotting trained
    session3goodTrained  ='Rat123_2017_03_17_17_52'
    #plotting stupid naive animal
    session3badNiave     ='Rat132_2017_05_26_13_34'
    #plotting stupid trained animal
    session3badTrained   ='Rat132_2017_06_09_14_06'
    #================================================
    
    # GRID 4: Trajectory examples
    
    #plotting naive
    trials4goodNaive=range(10,18)
    #plotting trained
    trials4goodTrained=range(50,59)
    #plotting stupid naive animal
    trials4badNiave=range(121,131)
    #plotting stupid trained animal
    trials4badTrained=range(70,80)
    
    

Plotting

In [ ]:
if "__file__" not in dir():
    figsize=(7.5,9)
    fig=plt.figure(figsize=figsize,dpi=600)
    
    #space for treadmill image
    
    ##########################################
    # 1: learning curve
    gs1= fig.add_gridspec(nrows=1, ncols=1, left=0.02, bottom=0.02, right=0.47, top=.18)
    ax1= fig.add_subplot(gs1[0])
    plot_dotted_learning_curve(ax1, root, animalList1, profile1, TaskParamToPlot1, stop_dayPlot1)
    ax1.set_ylabel('')
    
    
    ##########################################
    # 2: goal time change
    gs2= fig.add_gridspec(nrows=1, ncols=1, left=.53, bottom=0.02, right=0.98, top=.18)
    axes2=plot_goal_time_change(root, Profiles2, N2, GT2, badAnimals2, TaskParamToPlot2, wspace2, gs2[0], fig)
    axes2[0].set_ylabel('')
    
    
    ###########################################
    # 3: entrance time examples
    gs3= fig.add_gridspec(nrows=4, ncols=1, left=0.7, bottom=0.25, right=0.98, top=0.8)
    
    naiveAx3= fig.add_subplot(gs3[0])
    plot_trajectories_aligned_on_entrance_time(root, naiveAx3, session3goodNaive, Win3)
    
    trainedAx3= fig.add_subplot(gs3[1])
    plot_trajectories_aligned_on_entrance_time(root, trainedAx3, session3goodTrained, Win3)
    
    NaiveStupidAx3= fig.add_subplot(gs3[2])
    plot_trajectories_aligned_on_entrance_time(root, NaiveStupidAx3, session3badNiave, Win3)
    
    trainedStupidAx3= fig.add_subplot(gs3[3])
    plot_trajectories_aligned_on_entrance_time(root, trainedStupidAx3, session3badTrained, Win3)
    
    axes3=[naiveAx3, trainedAx3, NaiveStupidAx3, trainedStupidAx3]
    for ax in axes3:
        ax.yaxis.set_visible(False)
        ax.spines['left'].set_visible(False)
    
    
    ###########################################
    # 4: trajectory examples
    gs4= fig.add_gridspec(nrows=4, ncols=1, left=0.02, bottom=0.25, right=0.7, top=0.8)
    
    naiveAx4= fig.add_subplot(gs4[0])
    plot_consecutive_trajectories(root, session3goodNaive, trials4goodNaive, naiveAx4)
    naiveAx4.text(x=0, y=0, s=f'{session3goodNaive[:6]} naive')
    
    trainedAx4= fig.add_subplot(gs4[1])
    plot_consecutive_trajectories(root, session3goodTrained, trials4goodTrained, trainedAx4)
    trainedAx4.text(x=0, y=0, s=f'{session3goodTrained[:6]} trained')
    
    NaiveStupidAx4= fig.add_subplot(gs4[2])
    plot_consecutive_trajectories(root, session3badNiave, trials4badNiave, NaiveStupidAx4)
    NaiveStupidAx4.text(x=0, y=60, s=f'{session3badNiave[:6]} naive')
    
    trainedStupidAx4= fig.add_subplot(gs4[3])
    plot_consecutive_trajectories(root, session3badTrained, trials4badTrained, trainedStupidAx4)
    trainedStupidAx4.text(x=0, y=60, s=f'{session3badTrained[:6]} trained')
    
    axes4=[naiveAx4, trainedAx4, NaiveStupidAx4, trainedStupidAx4]
    timeMax=max([ax.get_xlim()[1] for ax in axes4])
    for ax in axes4:
        ax.set_xlim([0,timeMax])
        ax.xaxis.set_visible(False)
        ax.spines['bottom'].set_visible(False)
    
    trainedStupidAx4.xaxis.set_visible(True)
    trainedStupidAx4.spines['bottom'].set_visible(True)
    trainedStupidAx4.set_xlabel('Time (s)')
    
    
    
    ###############################################
    # 5: task rules
    gs5=fig.add_gridspec(nrows=1, ncols=1, left=.4, bottom=.85, right=0.98, top=0.98)
    axes5=plot_task_rules(fig, gs=gs5[0], hspace=.4, alpha=1, xmin=-2, xmax=16, et=(9,5), GT=7, maxP=11)
    
    

    
    
    
#     fig.savefig('/home/david/Pictures/ImmTrd.pdf', format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')